# Setup EMME / Python Library / Utitlity Functions

In [1]:
import inro.modeller as _m
import inro.emme.desktop as _d
import csv
import os
import multiprocessing
import numpy as np
import pandas as pd
import sqlite3
import datetime
import traceback as _traceback

dt = _d.app.connect()
de = dt.data_explorer()
db = de.active_database()
ebs = de.databases()

util = _m.Modeller().tool("translink.util")

In [2]:
def open_emmbank(target_title):
    # open scenario emmebank
    for eb in ebs:
        title = eb.title()
        if title == target_title:    
            eb.open()
            eb = _m.Modeller().emmebank
            scenario_ok = True
            break
        else:
            scenario_ok = False
    
    # Scenario emmebank open error check
    if not(scenario_ok):
        raise Exception(target_title + " Emmebank is not in the project")
    return eb

In [3]:
# make sure modeller is closed or it will print to the python console in there
for eb in ebs:
    print eb.title()

UBCx_Ph2_2035BAU_2020-12-31
UBCx_Ph2_2035Alt_1A_2020-12-31
Minimal Base Databank


In [4]:
def df_pivot(table, index, columns, values):
    
    table = pd.pivot(table, index=index,
                          columns=columns,values=values).reset_index().reset_index()
    table.columns = ["index"]+list(table.columns)[1:]
    table = table.drop(columns="index")
    return table

In [5]:
# print table in notebook results
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

# Scenarios and Assumptions Info

In [6]:
scenarios_input = pd.read_csv(os.path.join("..","EconomicAnalysis\EconomicAnalysisTool_Input.csv"))

# load alternative databanks
alternative_table = scenarios_input[["Alternative","Alternative Databank","Horizon"]]
bau_table = pd.DataFrame()
bau_table[["Alternative Databank","Horizon"]] = scenarios_input[["BAU Databank","Horizon"]]
bau_table["Alternative"] = "BAU"
scenarios_table = pd.DataFrame()
scenarios_table[["Scenario","Databank","Horizon"]] = pd.concat([bau_table,alternative_table], 
                                                               sort=False)[["Alternative","Alternative Databank","Horizon"]]
scenarios_table = scenarios_table.reset_index(drop=True)
scenarios_table

,Scenario,Databank,Horizon
0,BAU,UBCx_Ph2_2035BAU_2020-12-31,2035
1,Alt_1A,UBCx_Ph2_2035Alt_1A_2020-12-31,2035


# A. Customer Service & User Experience

### *) Average Transit Travel Time
AM Transit Travel Time from Arbutus to UBC [min]

In [9]:
fromlocation, fromTAZ = ["Arbutus", 23360]
tolocation, toTAZ = ["UBC1", 21060]

rounding = 0
eb = _m.Modeller().emmebank
ZoneList = util.get_matrix_numpy(eb, "mozoneindex", reshape=False).astype(int).tolist()

fromTAZ_index = ZoneList.index(fromTAZ)
toTAZ_index   = ZoneList.index(toTAZ)

table = ListTable()
table.append(['Scenario \ HorizonYear', '2017', '2035', '2050'])

transit_tt = scenarios_table.copy()
transit_tt["TransitTravelTime"] = -1
for index,row in transit_tt.iterrows():
    eb = open_emmbank(row["Databank"])
    
    # calculate bus travel time
    bus_time  = util.get_matrix_numpy(eb, "AmBusIvtt")
    bus_time += util.get_matrix_numpy(eb, "AmBusWait")
    bus_time += util.get_matrix_numpy(eb, "AmBusAux")
    bus_time += util.get_matrix_numpy(eb, "AmBusBoard")
    # calculate rail travel time
    rail_time  = util.get_matrix_numpy(eb, "AmRailIvtt")
    rail_time += util.get_matrix_numpy(eb, "AmRailIvttBus")
    rail_time += util.get_matrix_numpy(eb, "AmRailWait")
    rail_time += util.get_matrix_numpy(eb, "AmRailAux")
    rail_time += util.get_matrix_numpy(eb, "AmRailBoard")
    # calculate wce travel time
    wce_time  = util.get_matrix_numpy(eb, "AmWceIvtt")
    wce_time += util.get_matrix_numpy(eb, "AmWceIvttRail")
    wce_time += util.get_matrix_numpy(eb, "AmWceIvttBus")
    wce_time += util.get_matrix_numpy(eb, "AmWceWait")
    wce_time += util.get_matrix_numpy(eb, "AmWceAux")
    wce_time += util.get_matrix_numpy(eb, "AmWceBoard")
    # calulate transit travel time = min(bus, rail, wce)
    transit_time = np.minimum(bus_time, rail_time)
    transit_time = np.minimum(transit_time, wce_time)
    
    transit_tt.at[index,"TransitTravelTime"] = transit_time[fromTAZ_index][toTAZ_index]
    
transit_tt = df_pivot(transit_tt, "Scenario", "Horizon", "TransitTravelTime")
transit_tt

,Scenario,2035
0,Alt_1A,21
1,BAU,30


### A1) Reduce Transit Travel Times
Annual System-wide Transit Time Savings in Hours (for BAU/Existing demand)

In [8]:
#re-export transit time savings without perception factors
def ExportData(eb, Dict, filename):
    util = _m.Modeller().tool("translink.util")
        
    OutputPath = os.path.join(util.get_eb_path(_m.Modeller().emmebank), 'EconomicAnalysis')
    if not os.path.exists(OutputPath):
        os.makedirs(OutputPath)
    OutputFile = os.path.join(OutputPath, filename)
    np.savez_compressed(OutputFile, **Dict)

def rename_ROH_TransitTimeCost(eb, mf_number):
    #convert matrix name to npz export name
        
    matrix_name = eb.matrix("mf{}".format(mf_number)).name
    Time_of_Day = matrix_name[:2]
    if "Rail" in matrix_name:
        Class = "RAL"
    else:
        Class = matrix_name[2:5]
    export_name = Time_of_Day + "C" + Class + "9"
    return export_name.upper()

def Export_ROH_TimeCost(eb):
    util = _m.Modeller().tool("translink.util")
    with_previous_export = os.path.isfile(os.path.join(util.get_eb_path(eb), 'EconomicAnalysis', 'ROH_Time_TransitRealTime.npz'))
    if with_previous_export:
        return # do not re-export
    
    Time_Dict = {}
    # list Transit Fare matrix number to be exported
    matrix_list = []
    matrix_list += [5304, 5314, 5324] # AM/MD/PM Bus Fare
    matrix_list += [5505, 5515, 5525] # AM/MD/PM Rail Fare
    matrix_list += [5706, 5726] # AM/PM WCE Fare
    NoTAZ = len(util.get_matrix_numpy(eb, "zoneindex"))
    for mat_id in matrix_list:
        mat_name = "mf{}".format(mat_id)
        name_key = rename_ROH_TransitTimeCost(eb, mat_id)
        
        name_key = name_key[:2] + "M" + name_key[-4:]
        TimeMatrixCount = int(mat_name[-1])
        Time_Dict[name_key] = np.zeros((NoTAZ,NoTAZ))
        for TimeMatrixDigit in range(0,TimeMatrixCount):
            transit_perception_factor = 1.0
            time_mat_name = mat_name[:-1] + str(TimeMatrixDigit)
            Time_Dict[name_key] += util.get_matrix_numpy(eb, time_mat_name)*transit_perception_factor
            
    ExportData(eb, Time_Dict, "ROH_Time_TransitRealTime.npz")
    
for index, row in scenarios_input.iterrows():
    ProjectScenarioFolder = row["Alternative Databank"]
    BAUScenarioFolder = row["BAU Databank"]
    
    eb = open_emmbank(ProjectScenarioFolder)
    eb = _m.Modeller().emmebank
    Export_ROH_TimeCost(eb)
    eb = open_emmbank(BAUScenarioFolder)
    eb = _m.Modeller().emmebank
    Export_ROH_TimeCost(eb)

In [13]:
transit_tt_savings = scenarios_input.copy()
transit_tt_savings["transit_time_savings[hours]"]=-1

result = ['Person-Hours by Transit Mode','N/A', 'N/A']
rounding = -2

# mode_list = {mode_category: mode_group [mode, mode, mode]}
mode_list = {"Auto": ["SOV1", "SOV2", "SOV3", "SOV4", "HOV1", "HOV2", "HOV3"],
             "Transit": ["BUS", "RAL", "WCE"],
             "Light_Truck": ["LGV"], 
             "Heavy_Truck": ["HGV"]}

expansion_factors = {"SOV": [3.44, 8.41, 3.95],
                     "HOV": [1.51, 8.58, 5.32],
                     "BUS": [2.54, 9.44, 2.57],
                     "RAL": [2.53, 9.54, 2.92],
                     "WCE": [3.34,    0, 2.02],
                     "LGV": [3.59, 5.63, 6.17],
                     "HGV": [4.88, 5.43, 6.36]}

DailyToAnnual_factors = {"SOV":335,"HOV":335,"BUS":299,"RAL":331,"WCE":224,"LGV":313,"HGV":276}

eb = _m.Modeller().emmebank
ZoneList = util.get_matrix_numpy(eb, "mozoneindex", reshape=False).astype(int).tolist()
NoTAZ = len(ZoneList)
project_dir = os.path.abspath(os.path.join(eb.path,"../.."))

TAZ_Result = {}
TAZ_Result["TAZ"] = ZoneList
    
for index, row in transit_tt_savings.iterrows():
    ProjectScenarioFolder = row["Alternative Databank"]
    BAUScenarioFolder = row["BAU Databank"]
    
    Base_Demand= np.load(project_dir+"\\"+BAUScenarioFolder+"\\EconomicAnalysis\\ROH_Demand.npz")
    #Base_Time  = np.load(project_dir+BAUScenarioFolder+"/EconomicAnalysis/ROH_Time.npz")
    Base_Time= np.load(project_dir+"\\"+BAUScenarioFolder+"\\EconomicAnalysis\\ROH_Time_TransitRealTime.npz")
    Altr_Demand= np.load(project_dir+"\\"+ProjectScenarioFolder+"\\EconomicAnalysis\\ROH_Demand.npz")
    #Altr_Time  = np.load(project_dir+ProjectScenarioFolder+"/EconomicAnalysis/ROH_Time.npz")
    Altr_Time= np.load(project_dir+"\\"+ProjectScenarioFolder+"\\EconomicAnalysis\\ROH_Time_TransitRealTime.npz")
    
    Annual_Time_BenefitMinutes = 0
    
    for mode in ["BUS", "RAL"]:
        mode = mode + "9"
        
        Time_Benefit_AM = np.minimum(Base_Demand["AMT"+mode], Altr_Demand["AMT"+mode]) * (Base_Time["AMM"+mode] - Altr_Time["AMM"+mode])
        Time_Benefit_MD = np.minimum(Base_Demand["MDT"+mode], Altr_Demand["MDT"+mode]) * (Base_Time["MDM"+mode] - Altr_Time["MDM"+mode])
        Time_Benefit_PM = np.minimum(Base_Demand["PMT"+mode], Altr_Demand["PMT"+mode]) * (Base_Time["PMM"+mode] - Altr_Time["PMM"+mode])
        
        AM_Fac, MD_Fac, PM_Fac = expansion_factors[mode[:3]]
        AnnualFactor = DailyToAnnual_factors[mode[:3]]
        Annual_Time_BenefitMinutes += (Time_Benefit_AM * AM_Fac + Time_Benefit_MD * MD_Fac + Time_Benefit_PM * PM_Fac)*AnnualFactor
    
    result =  int(round((Annual_Time_BenefitMinutes.sum())/60,rounding)) #convert minutes to hours
    transit_tt_savings.at[index,"transit_time_savings[hours]"] = result
    
    scenario_name = row["Alternative"]+"_"+str(row["Horizon"])
    TAZ_Result[scenario_name] = np.sum(Annual_Time_BenefitMinutes,axis=1)

TAZ_Result=pd.DataFrame.from_dict(TAZ_Result,orient='index').transpose()
TAZ_Result_Header = ['TAZ']  + [col for col in TAZ_Result if col != 'TAZ']
TAZ_Result=TAZ_Result[TAZ_Result_Header]
TAZ_Result.to_csv("A1.csv",index=False)
    
transit_tt_savings = df_pivot(transit_tt_savings, "Alternative", "Horizon", "transit_time_savings[hours]")
transit_tt_savings

,Alternative,2035
0,Alt_1A,3039500


### A3) Improve Safety and Security
2035/2050 Reduction in accidents and collisions (vkt-based)

In [15]:
def compute_network_based_vkt(sc):
    util = _m.Modeller().tool("translink.util")
    calc_link = _m.Modeller().tool("inro.emme.network_calculation.network_calculator")
    
    spec = {"result": None, "expression": "(@sov1+@sov2+@sov3+@sov4)*length", "selections": {"link": "all"}, "aggregation": None, "type": "NETWORK_CALCULATION"}
    SOV_VKT = calc_link(spec)["sum"]
    spec = {"result": None, "expression": "(@hov1+@hov2+@hov3)*length", "selections":  {"link": "all"}, "aggregation": None, "type": "NETWORK_CALCULATION"}
    HOV_VKT = calc_link(spec)["sum"]
    spec = {"result": None, "expression": "(@lgvol/1.5)*length", "selections":  {"link": "all"}, "aggregation": None, "type": "NETWORK_CALCULATION"}
    LGV_VKT = calc_link(spec)["sum"]
    spec = {"result": None, "expression": "(@hgvol/2.5)*length", "selections":  {"link": "all"}, "aggregation": None, "type": "NETWORK_CALCULATION"}
    HGV_VKT = calc_link(spec)["sum"]
    return SOV_VKT, HOV_VKT, LGV_VKT, HGV_VKT

rounding = -2
        
TimeofdayToDaily_factors = {"SOV": [3.44, 8.41, 3.95], #AM,MD,PM
                            "HOV": [1.51, 8.58, 5.32],
                            "LGV": [3.59, 5.63, 6.17],
                            "HGV": [4.88, 5.43, 6.36]}
DailyToAnnual_factors = [335,335,313,276] #SOV,HOV,LGV,HGV

vkt_table = scenarios_input.copy()
vkt_table["Daily"] = -1.0
vkt_table["Annual"] = -1.0
vkt_table["AM Peak"] = -1.0
vkt_table["PM Peak"] = -1.0

vkt_percapita_table = scenarios_input.copy()
vkt_percapita_table["Daily Reduction per Capita"] = -1.0
vkt_percapita_table["Annual Reduction per Capita"] = -1.0

rounding = -2

for index, row in vkt_table.iterrows():
    ProjectScenario = row["Alternative Databank"]
    BAUScenario = row["BAU Databank"]
    
    #get Project VKT Dictionary
    eb = open_emmbank(ProjectScenario)
    am_scenario = int(eb.matrix("ms2").data)
    md_scenario = int(eb.matrix("ms3").data)
    pm_scenario = int(eb.matrix("ms4").data)
    Project_VKT = {}
    db = de.active_database()
    for sc in db.scenarios():
        scenario_number = sc.number()
        if scenario_number in [am_scenario, md_scenario, pm_scenario]:
            de.replace_primary_scenario(sc)
            TimeOfDay_Index = [am_scenario, md_scenario, pm_scenario].index(scenario_number)
            TimeOfDays = ["AM","MD","PM"]
            Project_VKT[TimeOfDays[TimeOfDay_Index]]=compute_network_based_vkt(sc)
    
    #get BAU VKT Dictionary
    eb = open_emmbank(BAUScenario)
    am_scenario = int(eb.matrix("ms2").data)
    md_scenario = int(eb.matrix("ms3").data)
    pm_scenario = int(eb.matrix("ms4").data)
    BAU_VKT = {}
    db = de.active_database()
    for sc in db.scenarios():
        scenario_number = sc.number()
        if scenario_number in [am_scenario, md_scenario, pm_scenario]:
            de.replace_primary_scenario(sc)
            TimeOfDay_Index = [am_scenario, md_scenario, pm_scenario].index(scenario_number)
            TimeOfDay = ["AM","MD","PM"]
            BAU_VKT[TimeOfDay[TimeOfDay_Index]]=compute_network_based_vkt(sc)
    
    population = util.get_matrix_numpy(eb, "mo10").sum()
    
    #Compute AM/MD/PM/Daily/Annual VKT
    VKT_Summary = []
    for VKT_Dict in [BAU_VKT, Project_VKT]:
        VKT_ScenarioSummary = []
        DailyVKT = 0
        AnnualVKT = 0
        for TimePeriod in ["AM","MD","PM"]:
            SOV,HOV,LGV,HGV = VKT_Dict[TimePeriod]
            VKT_ScenarioSummary.append(SOV+HOV+LGV+HGV)
            
            TimeOfDay_Index = ["AM","MD","PM"].index(TimePeriod)
            DailyVKT += SOV*TimeofdayToDaily_factors["SOV"][TimeOfDay_Index]
            DailyVKT += HOV*TimeofdayToDaily_factors["HOV"][TimeOfDay_Index]
            DailyVKT += LGV*TimeofdayToDaily_factors["LGV"][TimeOfDay_Index]
            DailyVKT += HGV*TimeofdayToDaily_factors["HGV"][TimeOfDay_Index]
            
            AnnualVKT += SOV*TimeofdayToDaily_factors["SOV"][TimeOfDay_Index]*DailyToAnnual_factors[0]
            AnnualVKT += HOV*TimeofdayToDaily_factors["HOV"][TimeOfDay_Index]*DailyToAnnual_factors[1]
            AnnualVKT += LGV*TimeofdayToDaily_factors["LGV"][TimeOfDay_Index]*DailyToAnnual_factors[2]
            AnnualVKT += HGV*TimeofdayToDaily_factors["HGV"][TimeOfDay_Index]*DailyToAnnual_factors[3]
            
        VKT_ScenarioSummary.append(DailyVKT)
        VKT_ScenarioSummary.append(AnnualVKT)
        VKT_Summary.append(VKT_ScenarioSummary)
            
    VKT_Reduction = [BAU_VKT_i - Project_VKT_i for BAU_VKT_i, Project_VKT_i in zip (VKT_Summary[0], VKT_Summary[1])]
    VKT_Reduction_percapita = [round(x/population, 2) for x in VKT_Reduction]
    VKT_Reduction = [int(round(x, rounding)) for x in VKT_Reduction]
    
    vkt_table.at[index,"Daily"] = VKT_Reduction[3]
    vkt_table.at[index,"Annual"] = VKT_Reduction[4]
    vkt_table.at[index,"AM Peak"] = VKT_Reduction[0]
    vkt_table.at[index,"PM Peak"] = VKT_Reduction[2]
    
    vkt_percapita_table.at[index,"Daily Reduction per Capita"] = VKT_Reduction_percapita[3]
    vkt_percapita_table.at[index,"Annual Reduction per Capita"] = VKT_Reduction_percapita[4]
    
vkt_table = vkt_table.drop(columns=["Alternative Databank","BAU Databank"])
vkt_percapita_table = vkt_percapita_table.drop(columns=["Alternative Databank","BAU Databank"])

In [16]:
collision_factor = 0.65/1000000

collision_table = vkt_table.copy()
collision_table["Annual Collision Reduction"] = collision_table["Annual"]*collision_factor
collision_table = collision_table.drop(columns=["Daily","Annual","AM Peak","PM Peak"])
collision_table.to_csv("A3.csv",index=False)
collision_table

,Alternative,Horizon,Annual Collision Reduction
0,Alt_1A,2035,35.24118


### A4a) Baseline Auto Travel Time Savings
Annual Auto Travel Time Savings in Hours

In [18]:
auto_tt_savings = scenarios_input.copy()
auto_tt_savings["auto_time_savings[hours]"]=-1

result = ['Person-Hours by Auto Mode','N/A', 'N/A']
rounding = -2
project_dir = os.path.abspath(os.path.join(eb.path,"../.."))

# mode_list = {mode_category: mode_group [mode, mode, mode]}
mode_list = {"Auto": ["SOV1", "SOV2", "SOV3", "SOV4", "HOV1", "HOV2", "HOV3"],
             "Transit": ["BUS", "RAL", "WCE"],
             "Light_Truck": ["LGV"], 
             "Heavy_Truck": ["HGV"]}

expansion_factors = {"SOV": [3.44, 8.41, 3.95],
                     "HOV": [1.51, 8.58, 5.32],
                     "BUS": [2.54, 9.44, 2.57],
                     "RAL": [2.53, 9.54, 2.92],
                     "WCE": [3.34,    0, 2.02],
                     "LGV": [3.59, 5.63, 6.17],
                     "HGV": [4.88, 5.43, 6.36]}

DailyToAnnual_factors = {"SOV":335,"HOV":335,"BUS":299,"RAL":331,"WCE":224,"LGV":313,"HGV":276}

TAZ_Result_auto = {}
TAZ_Result_auto["TAZ"] = ZoneList

for index, row in auto_tt_savings.iterrows():
    ProjectScenarioFolder = row["Alternative Databank"]
    BAUScenarioFolder = row["BAU Databank"]
    
    Base_Demand= np.load(project_dir+"\\"+BAUScenarioFolder+"\\EconomicAnalysis/ROH_Demand.npz")
    Base_Time  = np.load(project_dir+"\\"+BAUScenarioFolder+"\\EconomicAnalysis/ROH_Time.npz")
    Altr_Demand= np.load(project_dir+"\\"+ProjectScenarioFolder+"\\EconomicAnalysis/ROH_Demand.npz")
    Altr_Time  = np.load(project_dir+"\\"+ProjectScenarioFolder+"\\EconomicAnalysis/ROH_Time.npz")
    
    Annual_Time_BenefitMinutes = 0
    
    for mode in mode_list["Auto"]:
        
        Time_Benefit_AM = (np.minimum(Base_Demand["AMT"+mode], Altr_Demand["AMT"+mode]) * (Base_Time["AMM"+mode] - Altr_Time["AMM"+mode]))
        Time_Benefit_MD = (np.minimum(Base_Demand["MDT"+mode], Altr_Demand["MDT"+mode]) * (Base_Time["MDM"+mode] - Altr_Time["MDM"+mode]))
        Time_Benefit_PM = (np.minimum(Base_Demand["PMT"+mode], Altr_Demand["PMT"+mode]) * (Base_Time["PMM"+mode] - Altr_Time["PMM"+mode]))
        
        AM_Fac, MD_Fac, PM_Fac = expansion_factors[mode[:3]]
        AnnualFactor = DailyToAnnual_factors[mode[:3]]
        Annual_Time_BenefitMinutes += (Time_Benefit_AM * AM_Fac + Time_Benefit_MD * MD_Fac + Time_Benefit_PM * PM_Fac)*AnnualFactor
    
    result=int(round((Annual_Time_BenefitMinutes.sum())/60,rounding))#convert minutes to hours
    auto_tt_savings.at[index,"auto_time_savings[hours]"] = result
    
    scenario_name = row["Alternative"]+"_"+str(row["Horizon"])
    TAZ_Result_auto[scenario_name] = np.sum(Annual_Time_BenefitMinutes,axis=1)

TAZ_Result_auto=pd.DataFrame.from_dict(TAZ_Result_auto,orient='index').transpose()
TAZ_Result_auto_Header = ['TAZ']  + [col for col in TAZ_Result_auto if col != 'TAZ']
TAZ_Result_auto=TAZ_Result_auto[TAZ_Result_auto_Header]
TAZ_Result_auto.to_csv("A4a.csv",index=False)

auto_tt_savings = df_pivot(auto_tt_savings, "Alternative", "Horizon", "auto_time_savings[hours]")
auto_tt_savings

,Alternative,2035
0,Alt_1A,1365100


### *) Reduction in Auto Delay (for each trip)
Reduction in congestion minutes from congestion minutes OD matrix <br>
AM SOV3

In [23]:
def run_congestedassignment(eb):
    eb = open_emmbank(eb)
    congestion_assignment = _m.Modeller().tool("translink.congestedassignment")
    am_scen = eb.scenario(int(eb.matrix("msAmScen").data))
    md_scen = eb.scenario(int(eb.matrix("msMdScen").data))
    pm_scen = eb.scenario(int(eb.matrix("msPmScen").data))
    congestion_assignment(eb, am_scen, md_scen, pm_scen, "timau", 1)
    
for index,row in scenarios_table.iterrows():
    run_congestedassignment(row["Databank"])


In [103]:
delay_reduction_table = []
header = ["alternative","horizon","from","to","auto delay reduction [min]"]

fromTAZ_List = [["Fleetwood", 63520], 
                ["Langley",   67080]]
toTAZ_List = [["SurreyCentral",       61440], 
              ["CommercialBroadway",  25420], 
              ["ProductionWay",       27520], 
              ["SFU",                 27160], 
              ["CoquitlamCityCentre", 32030]]
eb = _m.Modeller().emmebank
ZoneList = util.get_matrix_numpy(eb, "mozoneindex", reshape=False).astype(int).tolist()
project_dir = os.path.abspath(os.path.join(eb.path,"../.."))

table = ListTable()
for index, row in scenarios_input.iterrows():
    ScenarioFolder = row["Alternative Databank"]
    BAUFolder = row["BAU Databank"]
    
    Delay_Project  = np.load(project_dir+"\\"+ScenarioFolder+"\\EconomicAnalysis\\TIMECONG.npz")
    Delay_BAU  = np.load(project_dir+"\\"+BAUFolder+"\\EconomicAnalysis\\TIMECONG.npz")
            
    # each tolocation is a table row
    for tolocation, toTAZ in toTAZ_List:
        toTAZindex = ZoneList.index(toTAZ)
        for fromlocation, fromTAZ in fromTAZ_List:
            fromTAZindex = ZoneList.index(fromTAZ)
            
            #get reliability Table
            CongestionMinutes_Project = Delay_Project["AMSOVTIMECONGVOT3"][fromTAZindex,toTAZindex]
            CongestionMinutes_BAU = Delay_BAU["AMSOVTIMECONGVOT3"][fromTAZindex,toTAZindex]
            delay_reduction_table.append([str(row["Alternative"]), int(row["Horizon"]),
                                              fromlocation, tolocation,
                                              CongestionMinutes_BAU-CongestionMinutes_Project])            

delay_reduction_table = pd.DataFrame(delay_reduction_table, columns=header)

,alternative,horizon,from,to,auto delay reduction [min]
0,Alt_1A,2035,Fleetwood,SurreyCentral,0.012259
1,Alt_1A,2035,Langley,SurreyCentral,0.020348
2,Alt_1A,2035,Fleetwood,CommercialBroadway,0.461607
3,Alt_1A,2035,Langley,CommercialBroadway,0.505898
4,Alt_1A,2035,Fleetwood,ProductionWay,0.248219
5,Alt_1A,2035,Langley,ProductionWay,0.117008
6,Alt_1A,2035,Fleetwood,SFU,0.013744
7,Alt_1A,2035,Langley,SFU,-0.147667
8,Alt_1A,2035,Fleetwood,CoquitlamCityCentre,0.032675
9,Alt_1A,2035,Langley,CoquitlamCityCentre,0.064854


# B. Transportation

### B1) New Daily Transit Trips
Change in System-wide Daily Trips: Transit 
Total Transit Trips Project - BAU

In [13]:
DailyTripsbyMode = _m.Modeller().tool("translink.MAE_DailyTripsByMode")

rounding = -2

import sys #suppress print statements in daily mode share script
sys.stdout = open(os.devnull, 'w')

new_transit_trips = scenarios_input.copy()
new_transit_trips["net new transit trips"] = -1
for index, row in new_transit_trips.iterrows():
    ProjectScenario = row["Alternative Databank"]
    BAUScenario = row["BAU Databank"]
    
    eb = open_emmbank(ProjectScenario)
    trip_total, trips_by_mode_Dict = DailyTripsbyMode.getDemand(eb)
    ProjectScenario_TransitTrips = trips_by_mode_Dict["Transit"]
    
    eb = open_emmbank(BAUScenario)
    trip_total, trips_by_mode_Dict = DailyTripsbyMode.getDemand(eb)
    BAUScenario_TransitTrips = trips_by_mode_Dict["Transit"]
    
    result = int(round(ProjectScenario_TransitTrips-BAUScenario_TransitTrips,rounding))
    new_transit_trips.at[index,"net new transit trips"] = result
        
sys.stdout = sys.__stdout__ # change print setting to default

new_transit_trips = df_pivot(new_transit_trips, "Alternative", "Horizon", "net new transit trips")
new_transit_trips.to_csv("B1.csv",index=False)    
new_transit_trips

,Alternative,2035
0,Alt_1A,18300


### B2) Increase Sustainable Mode Share
Daily trips by mode, by origin and by destination

In [12]:
DailyTripsbyMode = _m.Modeller().tool("translink.MAE_DailyTripsByMode")

TAZ_Result_TripByMode = {}
TAZ_Result_TripByMode["TAZ"] = ZoneList

import sys #suppress print statements in daily mode share script
sys.stdout = open(os.devnull, 'w')

for index,row in scenarios_table.iterrows():
    eb = open_emmbank(row["Databank"])
    scenario_label = row["Scenario"]+"_"+str(row["Horizon"])
    
    TripsByMode = DailyTripsbyMode.getDemand(eb,export_vector=True)
    for key, value in TripsByMode.iteritems():
        TAZ_Result_TripByMode[scenario_label+"_"+key]=value

TAZ_Result_TripByMode=pd.DataFrame.from_dict(TAZ_Result_TripByMode,orient='index').transpose()
TAZ_Result_TripByMode_Header = ['TAZ']  + [col for col in TAZ_Result_TripByMode if col != 'TAZ']
TAZ_Result_TripByMode=TAZ_Result_TripByMode[TAZ_Result_TripByMode_Header]
TAZ_Result_TripByMode.to_csv("B2.csv",index=False)    
        
sys.stdout = sys.__stdout__ # change print setting to default

### *) Number of Trips by Active Modes
1) Incremental Trips by Active Modes to Access Transit against BAU <br>
2) Additional Kilometers Travelled by Active Modes to Access Transit

In [17]:
def getWalkTripsDistance(eb):
    eb = open_emmbank(eb)
    
    walkTrips = 0
    walkDistance = 0
    walkSpeed = 4.8 #km/hr - speed factor of walking mode
    
    matrix_list =  [["mf314","mf5302", "mf5303", "AM", "BUS"], #AM Bus
                    ["mf315","mf5503", "mf5504", "AM", "RAL"], #AM Rail
                    ["mf316","mf5704", "mf5705", "AM", "WCE"], #AM WCE
                    ["mf334","mf5312", "mf5313", "MD", "BUS"], #MD Bus
                    ["mf335","mf5513", "mf5514", "MD", "RAL"], #MD Rail
                    ["mf354","mf5322", "mf5323", "PM", "BUS"], #PM Bus
                    ["mf355","mf5523", "mf5524", "PM", "RAL"], #PM Rail
                    ["mf356","mf5724", "mf5725", "PM", "WCE"]] #PM WCE

    expansion_factors = {"SOV": [3.44, 8.41, 3.95],
                         "HOV": [1.51, 8.58, 5.32],
                         "BUS": [2.54, 9.44, 2.57],
                         "RAL": [2.53, 9.54, 2.92],
                         "WCE": [3.34,    0, 2.02],
                         "LGV": [3.59, 5.63, 6.17],
                         "HGV": [4.88, 5.43, 6.36]}
    DailyToAnnual_factors = {"SOV":335,"HOV":335,"BUS":299,"RAL":331,"WCE":224,"LGV":313,"HGV":276}
    
    for trips,auxillaryTime,boardings,TOD,Mode in matrix_list:
        TOD_Index = ["AM","MD","PM"].index(TOD)
        expansion = DailyToAnnual_factors[Mode] * expansion_factors[Mode][TOD_Index]
        
        mf_trips = util.get_matrix_numpy(eb, trips, reshape=False)
        mf_auxillaryTime = util.get_matrix_numpy(eb, auxillaryTime, reshape=False)
        mf_boardings = util.get_matrix_numpy(eb, boardings, reshape=False)
        
        walkTrips += mf_trips * (mf_boardings + 1) * expansion
        walkDistance += mf_trips * (walkSpeed * mf_auxillaryTime/60) * expansion #convert walk time from minutes to hours
    return walkTrips.sum(),walkDistance.sum()


active_trips = scenarios_input.copy()
active_trips["incremental active trips"] = -1
active_trips["additional active vkt"] = -1

rounding = -2

for index, row in active_trips.iterrows():
    ProjectScenario = row["Alternative Databank"]
    BAUScenario = row["BAU Databank"]
    
    WalkTrips_Project, WalkDistance_Project = getWalkTripsDistance(ProjectScenario)
    WalkTrips_BAU,     WalkDistance_BAU     = getWalkTripsDistance(BAUScenario)
    
    incremental_trips = int(round(WalkTrips_Project - WalkTrips_BAU,rounding))
    incremental_vkt = int(round(WalkDistance_Project - WalkDistance_BAU,rounding))
    
    active_trips.at[index,"incremental active trips"] = incremental_trips
    active_trips.at[index,"additional active vkt"] = incremental_vkt
    
active_trips = active_trips.drop(columns=["Alternative Databank","BAU Databank"])
active_trips

,Alternative,Horizon,incremental active trips,additional active vkt
0,Alt_1A,2035,16907400,16240800


# 3. Social, Community, Environment

### a) System-wide VKT Reduction Compared to BAU
Daily, Annual, AM, PM

In [93]:
vkt_table

,Alternative,Horizon,Daily,Annual,AM Peak,PM Peak
0,Alt_1A,2035,161800.0,54217200.0,16600.0,14000.0


### VKT Reduction Per Capita

In [94]:
vkt_percapita_table

,Alternative,Horizon,Daily Reduction per Capita,Annual Reduction per Capita
0,Alt_1A,2035,0.04,14.57


# 4. Economic Development

### a) Number of new direct and indirect jobs created compared to BAU in 3035 during construction (person-years of employment)
on hold

### b) WEBs - regional economic benefit and regional prosperity

In [25]:
# discussion?

### c) Goods and Market Impact
1) Daily Truck Travel Time Saving (Hours) <br>
2) Annual Truck Travel Time Saving in dollars

In [22]:
truck_table = scenarios_input.copy()
truck_table["Daily Truck Time Saving Hours"] = -1
truck_table["Annual Truck Time Savings [2019$]"] = -1

rounding1 = -1
rounding2 = -2
project_dir = os.path.abspath(os.path.join(eb.path,"../.."))

# mode_list = {mode_category: mode_group [mode, mode, mode]}
mode_list = {"Auto": ["SOV1", "SOV2", "SOV3", "SOV4", "HOV1", "HOV2", "HOV3"],
             "Transit": ["BUS", "RAL", "WCE"],
             "Light_Truck": ["LGV"], 
             "Heavy_Truck": ["HGV"]}

expansion_factors = {"SOV": [3.44, 8.41, 3.95],
                     "HOV": [1.51, 8.58, 5.32],
                     "BUS": [2.54, 9.44, 2.57],
                     "RAL": [2.53, 9.54, 2.92],
                     "WCE": [3.34,    0, 2.02],
                     "LGV": [3.59, 5.63, 6.17],
                     "HGV": [4.88, 5.43, 6.36]}

DailyToAnnual_factors = {"SOV":335,"HOV":335,"BUS":299,"RAL":331,"WCE":224,"LGV":313,"HGV":276}

for index, row in truck_table.iterrows():
    ProjectScenarioFolder = row["Alternative Databank"]
    BAUScenarioFolder = row["BAU Databank"]
    
    Base_Demand= np.load(project_dir+"\\"+BAUScenarioFolder+"\\EconomicAnalysis/ROH_Demand.npz")
    Base_Time  = np.load(project_dir+"\\"+BAUScenarioFolder+"\\EconomicAnalysis/ROH_Time.npz")
    Altr_Time  = np.load(project_dir+"\\"+ProjectScenarioFolder+"\\EconomicAnalysis/ROH_Time.npz")
    
    Time_Benefit_AM = 0
    Time_Benefit_MD = 0
    Time_Benefit_PM = 0
    Daily_Time_BenefitMinutes = 0
    Annual_Time_BenefitMinutes = 0
    
    for mode in ["LGV","HGV"]:
        mode = mode + "9"
        Time_Benefit_AM = (Base_Demand["AMT"+mode] * (Base_Time["AMM"+mode] - Altr_Time["AMM"+mode])).sum()
        Time_Benefit_MD = (Base_Demand["MDT"+mode] * (Base_Time["MDM"+mode] - Altr_Time["MDM"+mode])).sum()
        Time_Benefit_PM = (Base_Demand["PMT"+mode] * (Base_Time["PMM"+mode] - Altr_Time["PMM"+mode])).sum()
        
        AM_Fac, MD_Fac, PM_Fac = expansion_factors[mode[:3]]
        Daily_Time_BenefitMinutes += Time_Benefit_AM * AM_Fac + Time_Benefit_MD * MD_Fac + Time_Benefit_PM * PM_Fac
        
        AnnualFactor = DailyToAnnual_factors[mode[:3]]
        Annual_Time_BenefitMinutes += (Time_Benefit_AM * AM_Fac + Time_Benefit_MD * MD_Fac + Time_Benefit_PM * PM_Fac)*AnnualFactor
    
    result = int(round((Daily_Time_BenefitMinutes)/60,rounding1))#convert minutes to hours
    truck_table.at[index,"Daily Truck Time Saving Hours"] = result
    
    TruckValueOfTime = 31.25 # 2018 Dollars/hr
    DollarYearAdjust = 1 + 0.02 # Adjust to 2019 Dollar by Applying Inflation Rate (CPI)
    Annual_Time_BenefitDollar = DollarYearAdjust * TruckValueOfTime * Annual_Time_BenefitMinutes/60
    result = int(round(Annual_Time_BenefitDollar,rounding2))
    truck_table.at[index,"Annual Truck Time Savings [2019$]"] = result
    
truck_table = truck_table.drop(columns=["Alternative Databank","BAU Databank"])
truck_table    

,Alternative,Horizon,Daily Truck Time Saving Hours,Annual Truck Time Savings [2019$]
0,Alt_1A,2035,150,1412800


In [92]:
# transit volume on UBC Screenline
create_extra = _m.Modeller().tool("inro.emme.data.extra_attribute.create_extra_attribute")
tag_tool = _m.Modeller().tool("translink.RTM3Analytics.GeographicTagging")
root_worksheet_folder = dt.root_worksheet_folder()
link_table_path = root_worksheet_folder.find_item(["MAE","TransitVolume_UBCScreenline"])
link_table = link_table_path.open()
link_table.par("ExportColumnSeparator").set(",")

transit_sl_tt = []
for index,row in scenarios_table.iterrows():
    eb = open_emmbank(row["Databank"])
    
    # open the scenario
    am_scenario = int(eb.matrix("ms2").data)
    db = de.active_database()
    for sc in db.scenarios():
        scenario_number = sc.number()
        if scenario_number==am_scenario:
            de.replace_primary_scenario(sc)
    
            # tag UBC screenline
            create_extra(extra_attribute_type="LINK",
                         extra_attribute_name="@ubcscreenline",
                         extra_attribute_description="UBC Screenline",
                         overwrite=True)
            # tag surrey links
            linkattributeName = "@ubcscreenline"
            polygonfile = os.path.abspath(os.path.join(eb.path, "..", "..", 
                                          "Media", "MAE", "UBC_Screenlline.shp"))
            algorithm = "tag_max"
            excludeconnector = True
            default = 0
            polygonfield = "SL_NUM"
            scen = eb.scenario(am_scenario)
            tag_tool(scen,linkattributeName,polygonfile,polygonfield,
                     algorithm,excludeconnector,default)
            
            # save transit segment volume @voltravg
            export_filepath = row["Scenario"] + "_" + str(row["Horizon"]) + ".csv"
            export_filepath = os.path.join(os.getcwd(), export_filepath)
            print(export_filepath)
            link_table.export(export_filepath)
            
            # append horizon and scenario
            transitvolume_UBC = pd.read_csv(export_filepath)
            transitvolume_UBC["Horizon"] = row["Horizon"]
            transitvolume_UBC["Scenario"] = row["Scenario"]
            
            # filter the WB routes
            net = scen.get_network()
            for i,row_i in transitvolume_UBC.iterrows():
                transitvolume_UBC.at[i,"i_x"] = net.node(row_i["i"]).x
                transitvolume_UBC.at[i,"j_x"] = net.node(row_i["j"]).x
            
            # keep WB routes (ix>jx), drop EB routes
            transitvolume_UBC = transitvolume_UBC[transitvolume_UBC.i_x>transitvolume_UBC.j_x]
            
            transit_sl_tt.append(transitvolume_UBC)
            os.remove(export_filepath)

link_table.close()
transit_sl_tt = pd.concat(transit_sl_tt)[["Horizon","Scenario","Route","Volume"]].reset_index()
transit_sl_tt["Scenario"] = transit_sl_tt["Horizon"].apply(str)+"_"+transit_sl_tt["Scenario"]
transit_sl_tt = df_pivot(transit_sl_tt, "Scenario","Route","Volume").fillna(0)
transit_sl_tt["Total_WB"] = transit_sl_tt.iloc[:, 1:].sum(axis=1)
transit_sl_tt

E:\2121-00651_UBCx_MAE\Scripts\BAU_2035.csv
E:\2121-00651_UBCx_MAE\Scripts\Alt_1A_2035.csv


,Scenario,004W1X,009W1A,014W1A,025W1X,033W1X,044W1X,049W1X,084W1X,099W3X,258S1A,258S2A,480W1X,932W1X,974W1X,Total_WB
0,2035_Alt_1A,34.32,79.58,95.62,300.28,110.66,441.46,377.52,162.84,0.00,39.22,39.27,187.91,7943.0,434.71,10246.39
1,2035_BAU,152.84,289.52,273.72,690.97,288.80,920.57,781.38,715.47,2924.65,53.34,52.61,388.66,0.0,1146.57,8679.10


In [21]:
# summarize csv files into one excel file
csv_list = ["INFO"] + pd.read_csv("INFO.csv")["Item"].to_list()

writer = pd.ExcelWriter('UBCx_MAE.xlsx', engine='xlsxwriter')
for sheetname in csv_list:
    try:
        pd.read_csv(sheetname+".csv").to_excel(writer, sheet_name=sheetname, index=False)
    except:
        print(sheetname+".csv")

writer.save()